In [ ]:
#installations
#!pip install --quiet SimpleITK
#Anaconda Powershell
#:L
#conda activate data_processing
#jupyter serverextension enable -- py jupyter_http_over_ws
#jupyter notebook --NotebookApp.allow_origin='https://colab.research.google.com' --port=8892 --NotebookApp.port_retries=0

In [ ]:
#imports
import os
import glob
import torch
import pandas as pd
import torch.nn as nn
from einops import rearrange
import sys
#this path must be specified
sys.path.append('AnishSalvi/ImageRx/PAD-Net/vRAD_Dataset_Inputs/Transformer_Explainability')
from modules.layers_ours import *
from baselines.ViT.helpers import load_pretrained
from baselines.ViT.weight_init import trunc_normal_
from baselines.ViT.layer_helpers import to_2tuple
from baselines.ViT.ViT_explanation_generator import LRP
#import matplotlib.pyplot as plt
import numpy as np
import SimpleITK as sitk
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import json
import sklearn
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
#check if the gpu machine is available
if torch.cuda.is_available():
  device = 'cuda'
  gpu = torch.cuda.get_device_name(0)
  print('Device: ', gpu)
else:
  device = 'cpu'
  gpu = None
  print('Device', device)

Device:  NVIDIA GeForce RTX 3060


In [ ]:
#functions

def compute_rollout_attention(all_layer_matrices, start_layer=0):
    # adding residual consideration
    num_tokens = all_layer_matrices[0].shape[1]
    batch_size = all_layer_matrices[0].shape[0]
    eye = torch.eye(num_tokens).expand(batch_size, num_tokens, num_tokens).to(all_layer_matrices[0].device)
    all_layer_matrices = [all_layer_matrices[i] + eye for i in range(len(all_layer_matrices))]
    # all_layer_matrices = [all_layer_matrices[i] / all_layer_matrices[i].sum(dim=-1, keepdim=True)
    #                       for i in range(len(all_layer_matrices))]
    joint_attention = all_layer_matrices[start_layer]
    for i in range(start_layer+1, len(all_layer_matrices)):
        joint_attention = all_layer_matrices[i].bmm(joint_attention)
    return joint_attention

class Mlp(nn.Module):
    def __init__(self, in_features, hidden_features=None, out_features=None, drop=0.):
        super().__init__()
        out_features = out_features or in_features
        hidden_features = hidden_features or in_features
        self.fc1 = Linear(in_features, hidden_features)
        self.act = GELU()
        self.fc2 = Linear(hidden_features, out_features)
        self.drop = Dropout(drop)

    def forward(self, x):
        x = self.fc1(x)
        x = self.act(x)
        x = self.drop(x)
        x = self.fc2(x)
        x = self.drop(x)
        return x

    def relprop(self, cam, **kwargs):
        cam = self.drop.relprop(cam, **kwargs)
        cam = self.fc2.relprop(cam, **kwargs)
        cam = self.act.relprop(cam, **kwargs)
        cam = self.fc1.relprop(cam, **kwargs)
        return cam


class Attention(nn.Module):
    def __init__(self, dim, num_heads=8, qkv_bias=False,attn_drop=0., proj_drop=0.):
        super().__init__()
        self.num_heads = num_heads
        head_dim = dim // num_heads
        # NOTE scale factor was wrong in my original version, can set manually to be compat with prev weights
        self.scale = head_dim ** -0.5

        # A = Q*K^T
        self.matmul1 = einsum('bhid,bhjd->bhij')
        # attn = A*V
        self.matmul2 = einsum('bhij,bhjd->bhid')

        self.qkv = Linear(dim, dim * 3, bias=qkv_bias)
        self.attn_drop = Dropout(attn_drop)
        self.proj = Linear(dim, dim)
        self.proj_drop = Dropout(proj_drop)
        self.softmax = Softmax(dim=-1)

        self.attn_cam = None
        self.attn = None
        self.v = None
        self.v_cam = None
        self.attn_gradients = None

    def get_attn(self):
        return self.attn

    def save_attn(self, attn):
        self.attn = attn

    def save_attn_cam(self, cam):
        self.attn_cam = cam

    def get_attn_cam(self):
        return self.attn_cam

    def get_v(self):
        return self.v

    def save_v(self, v):
        self.v = v

    def save_v_cam(self, cam):
        self.v_cam = cam

    def get_v_cam(self):
        return self.v_cam

    def save_attn_gradients(self, attn_gradients):
        self.attn_gradients = attn_gradients

    def get_attn_gradients(self):
        return self.attn_gradients

    #this may cause issues for 3d? or does it act upoon the patches

    def forward(self, x):
        b, n, _, h = *x.shape, self.num_heads
        qkv = self.qkv(x)
        q, k, v = rearrange(qkv, 'b n (qkv h d) -> qkv b h n d', qkv=3, h=h)

        self.save_v(v)

        dots = self.matmul1([q, k]) * self.scale

        attn = self.softmax(dots)
        attn = self.attn_drop(attn)

        self.save_attn(attn)
        attn.register_hook(self.save_attn_gradients)

        out = self.matmul2([attn, v])
        out = rearrange(out, 'b h n d -> b n (h d)')

        out = self.proj(out)
        out = self.proj_drop(out)
        return out

    def relprop(self, cam, **kwargs):
        cam = self.proj_drop.relprop(cam, **kwargs)
        cam = self.proj.relprop(cam, **kwargs)
        cam = rearrange(cam, 'b n (h d) -> b h n d', h=self.num_heads)

        # attn = A*V
        (cam1, cam_v)= self.matmul2.relprop(cam, **kwargs)
        cam1 /= 2
        cam_v /= 2

        self.save_v_cam(cam_v)
        self.save_attn_cam(cam1)

        cam1 = self.attn_drop.relprop(cam1, **kwargs)
        cam1 = self.softmax.relprop(cam1, **kwargs)

        # A = Q*K^T
        (cam_q, cam_k) = self.matmul1.relprop(cam1, **kwargs)
        cam_q /= 2
        cam_k /= 2

        cam_qkv = rearrange([cam_q, cam_k, cam_v], 'qkv b h n d -> b n (qkv h d)', qkv=3, h=self.num_heads)

        return self.qkv.relprop(cam_qkv, **kwargs)


class Block(nn.Module):

    def __init__(self, dim, num_heads, mlp_ratio=4., qkv_bias=False, drop=0., attn_drop=0.):
        super().__init__()
        self.norm1 = LayerNorm(dim, eps=1e-6)
        self.attn = Attention(
            dim, num_heads=num_heads, qkv_bias=qkv_bias, attn_drop=attn_drop, proj_drop=drop)
        self.norm2 = LayerNorm(dim, eps=1e-6)
        mlp_hidden_dim = int(dim * mlp_ratio)
        self.mlp = Mlp(in_features=dim, hidden_features=mlp_hidden_dim, drop=drop)

        self.add1 = Add()
        self.add2 = Add()
        self.clone1 = Clone()
        self.clone2 = Clone()

    def forward(self, x):
        x1, x2 = self.clone1(x, 2)
        x = self.add1([x1, self.attn(self.norm1(x2))])
        x1, x2 = self.clone2(x, 2)
        x = self.add2([x1, self.mlp(self.norm2(x2))])
        return x

    def relprop(self, cam, **kwargs):
        (cam1, cam2) = self.add2.relprop(cam, **kwargs)
        cam2 = self.mlp.relprop(cam2, **kwargs)
        cam2 = self.norm2.relprop(cam2, **kwargs)
        cam = self.clone2.relprop((cam1, cam2), **kwargs)

        (cam1, cam2) = self.add1.relprop(cam, **kwargs)
        cam2 = self.attn.relprop(cam2, **kwargs)
        cam2 = self.norm1.relprop(cam2, **kwargs)
        cam = self.clone1.relprop((cam1, cam2), **kwargs)
        return cam

#3D patch embedder
class PatchEmbed3D(nn.Module):
  """ Image to Patch Embedding
  """
  def __init__(self, img_size = (200, 100, 150), patch_size = (20, 10, 15), in_chans = 1, embed_dim = 768):
    super().__init__()
    img_size = to_2tuple(img_size)
    patch_size = to_2tuple(patch_size)
    num_patches = (img_size[2] // patch_size[2]) * (img_size[1] // patch_size[1]) * (img_size[0] // patch_size[0])
    self.img_size = img_size
    self.patch_size = patch_size
    self.num_patches = num_patches
    self.proj = torch.nn.Conv3d(in_chans, embed_dim, kernel_size = patch_size, stride = patch_size)

  def forward(self, x):
    B, C, H, W, D = x.shape
    # FIXME look at relaxing size constraints
    assert H == self.img_size[0] and W == self.img_size[1] and D == self.img_size[2], \
      f"Input image size ({H}*{W}*{D}) doesn't match model ({self.img_size[0]} * {self.img_size[1]} * {self.img_size[2]})."
    x = self.proj(x).flatten(2).transpose(1, 2)
    return x

  def relprop(self, cam, **kwargs):
    cam = cam.transpose(1,2) #why transpose?
    cam = cam.reshape(cam.shape[0], cam.shape[1], cam.reshape[2], #is the cam in 3d as well?
                      (self.img_size[0] // self.patch_size[0]),
                      (self.img_size[1] // self.patch_size[1]),
                      (self.img_size[2] // self.patch_size[2])
                      )
    return self.proj.relprop(cam, **kwargs)


class VisionTransformer(nn.Module):
    """ Vision Transformer with support for patch or hybrid CNN input stage
    """
    def __init__(self, img_size=(200, 100, 150), patch_size=(20, 10, 15), in_chans=1, num_classes=2, embed_dim=768, depth=12,
                 num_heads=12, mlp_ratio=4., qkv_bias=False, mlp_head=False, drop_rate=0., attn_drop_rate=0.):
        super().__init__()
        self.num_classes = num_classes
        self.num_features = self.embed_dim = embed_dim  # num_features for consistency with other models
        self.patch_embed = PatchEmbed3D(
                img_size=img_size, patch_size=patch_size, in_chans=in_chans, embed_dim=embed_dim)
        num_patches = self.patch_embed.num_patches

        self.pos_embed = nn.Parameter(torch.zeros(1, num_patches + 1, embed_dim))
        self.cls_token = nn.Parameter(torch.zeros(1, 1, embed_dim))

        self.blocks = nn.ModuleList([
            Block(
                dim=embed_dim, num_heads=num_heads, mlp_ratio=mlp_ratio, qkv_bias=qkv_bias,
                drop=drop_rate, attn_drop=attn_drop_rate)
            for i in range(depth)])

        self.norm = LayerNorm(embed_dim)
        if mlp_head:
            # paper diagram suggests 'MLP head', but results in 4M extra parameters vs paper
            self.head = Mlp(embed_dim, int(embed_dim * mlp_ratio), num_classes)
        else:
            # with a single Linear layer as head, the param count within rounding of paper
            self.head = Linear(embed_dim, num_classes)

        # FIXME not quite sure what the proper weight init is supposed to be,
        # normal / trunc normal w/ std == .02 similar to other Bert like transformers
        trunc_normal_(self.pos_embed, std=.02)  # embeddings same as weights?
        trunc_normal_(self.cls_token, std=.02)
        self.apply(self._init_weights)

        self.pool = IndexSelect()
        self.add = Add()

        self.inp_grad = None

    def save_inp_grad(self,grad):
        self.inp_grad = grad

    def get_inp_grad(self):
        return self.inp_grad


    def _init_weights(self, m):
        if isinstance(m, nn.Linear):
            trunc_normal_(m.weight, std=.02)
            if isinstance(m, nn.Linear) and m.bias is not None:
                nn.init.constant_(m.bias, 0)
        elif isinstance(m, nn.LayerNorm):
            nn.init.constant_(m.bias, 0)
            nn.init.constant_(m.weight, 1.0)

    @property
    def no_weight_decay(self):
        return {'pos_embed', 'cls_token'}

    def forward(self, x):
        B = x.shape[0]
        x = self.patch_embed(x)

        cls_tokens = self.cls_token.expand(B, -1, -1)  # stole cls_tokens impl from Phil Wang, thanks
        x = torch.cat((cls_tokens, x), dim=1)
        x = self.add([x, self.pos_embed])

        x.register_hook(self.save_inp_grad)

        for blk in self.blocks:
            x = blk(x)

        x = self.norm(x)
        x = self.pool(x, dim=1, indices=torch.tensor(0, device=x.device))
        x = x.squeeze(1)
        x = self.head(x)
        return x

    def relprop(self, cam=None,method="transformer_attribution", is_ablation=False, start_layer=0, **kwargs):
        # print(kwargs)
        # print("conservation 1", cam.sum())
        cam = self.head.relprop(cam, **kwargs)
        cam = cam.unsqueeze(1)
        cam = self.pool.relprop(cam, **kwargs)
        cam = self.norm.relprop(cam, **kwargs)
        for blk in reversed(self.blocks):
            cam = blk.relprop(cam, **kwargs)

        # print("conservation 2", cam.sum())
        # print("min", cam.min())

        if method == "full":
            (cam, _) = self.add.relprop(cam, **kwargs)
            cam = cam[:, 1:]
            cam = self.patch_embed.relprop(cam, **kwargs)
            # sum on channels
            cam = cam.sum(dim=1)
            return cam

        elif method == "rollout":
            # cam rollout
            attn_cams = []
            for blk in self.blocks:
                attn_heads = blk.attn.get_attn_cam().clamp(min=0)
                avg_heads = (attn_heads.sum(dim=1) / attn_heads.shape[1]).detach()
                attn_cams.append(avg_heads)
            cam = compute_rollout_attention(attn_cams, start_layer=start_layer)
            cam = cam[:, 0, 1:]
            return cam

        # our method, method name grad is legacy
        elif method == "transformer_attribution" or method == "grad":
            cams = []
            for blk in self.blocks:
                grad = blk.attn.get_attn_gradients()
                cam = blk.attn.get_attn_cam()
                cam = cam[0].reshape(-1, cam.shape[-1], cam.shape[-1])
                grad = grad[0].reshape(-1, grad.shape[-1], grad.shape[-1])
                cam = grad * cam
                cam = cam.clamp(min=0).mean(dim=0)
                cams.append(cam.unsqueeze(0))
            rollout = compute_rollout_attention(cams, start_layer=start_layer)
            cam = rollout[:, 0, 1:]
            return cam

        elif method == "last_layer":
            cam = self.blocks[-1].attn.get_attn_cam()
            cam = cam[0].reshape(-1, cam.shape[-1], cam.shape[-1])
            if is_ablation:
                grad = self.blocks[-1].attn.get_attn_gradients()
                grad = grad[0].reshape(-1, grad.shape[-1], grad.shape[-1])
                cam = grad * cam
            cam = cam.clamp(min=0).mean(dim=0)
            cam = cam[0, 1:]
            return cam

        elif method == "last_layer_attn":
            cam = self.blocks[-1].attn.get_attn()
            cam = cam[0].reshape(-1, cam.shape[-1], cam.shape[-1])
            cam = cam.clamp(min=0).mean(dim=0)
            cam = cam[0, 1:]
            return cam

        elif method == "second_layer":
            cam = self.blocks[1].attn.get_attn_cam()
            cam = cam[0].reshape(-1, cam.shape[-1], cam.shape[-1])
            if is_ablation:
                grad = self.blocks[1].attn.get_attn_gradients()
                grad = grad[0].reshape(-1, grad.shape[-1], grad.shape[-1])
                cam = grad * cam
            cam = cam.clamp(min=0).mean(dim=0)
            cam = cam[0, 1:]
            return cam


def _conv_filter(state_dict, patch_size=30):
    """ convert patch embedding weight from manual patchify + linear proj to conv"""
    out_dict = {}
    for k, v in state_dict.items():
        if 'patch_embed.proj.weight' in k:
            v = v.reshape((v.shape[0], 3, patch_size, patch_size))
        out_dict[k] = v
    return out_dict

#save json file
def save_params(hyper_params, save_path):
  json_string = json.dumps(hyper_params)
  with open(save_path, 'w') as outfile:
    outfile.write(json_string)

#load the train params back in
def load_params(fpath):
  # Opening JSON file
  with open(fpath) as json_file:
    data = json.load(json_file)
  return data

#get the model
def get_model(model_config):
  #model
  model = VisionTransformer(
      img_size = model_config['img_size'],
      patch_size = model_config['patch_size'],
      in_chans = model_config['in_chans'],
      num_classes = model_config['num_classes'],
      embed_dim = model_config['embed_dim'],
      depth = model_config['depth'],
      num_heads = model_config['num_heads'],
      mlp_ratio = model_config['mlp_ratio'],
      qkv_bias = model_config['qkv_bias'],
      mlp_head = model_config['mlp_head'],
      drop_rate = model_config['drop_rate'],
      attn_drop_rate = model_config['attn_drop_rate']
  )
  #return
  return model

#load pretrained model
def load_pretrained_model(save_folder, device):
  #load
  model_config = load_params(save_folder + 'model_config.json')
  #weights
  if model_config['save_weights_only']:
    model = get_model(model_config)
    #important to do prior
    model.eval()
    #load weights
    model.load_state_dict(torch.load(model_config['save_folder'] + 'model_weights.pth', map_location = device))
  else:
    model = torch.load(model_config['save_folder'] + 'model.pth', map_location = device)
  #send
  model.eval()
  model.to(device)
  #return
  return model, model_config


#Dataset
class CustomImageDataset(Dataset):
    def __init__(self, df, col_image):
      #params
      self.df = df
      self.col_image = col_image

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
      #row
      row = self.df.iloc[idx]

      #read image
      image = torch.Tensor(np.expand_dims(sitk.GetArrayFromImage(sitk.ReadImage(row[self.col_image])), axis = 0))

      #return
      return image

#split
def split_given_size(a, size):
  return np.split(a, np.arange(size, len(a), size))

#determine dataset splitting
def get_split(row, model_config):
  if row['KFold'] in model_config['val_kfolds']:
    dataset = 'VAL'
  else:
    dataset = 'TRAIN'
  #return
  return dataset

#function determine classification performance
def get_performance(df, model_config):
  #labels2class
  encoding = remove_repeat_sort(dict(zip(df['Annotation'],
                                           df[model_config['col_label']])))
  #store info
  model_config['encoding'] = encoding

  #iterate
  for split in set(df['Split']):
    #split data
    df_data = df[df['Split'] == split]
    #report
    config = sklearn.metrics.classification_report(df_data[model_config['col_label']].to_numpy().astype(float),
                                                   df_data['Class_Prediction'].to_numpy(), target_names = list(encoding.keys()),
                                                   output_dict = True)

    #save
    model_config[split + '_performance'] = config
    #plot confusion matrix
    plot_confusion_matrix(df_data[model_config['col_label']].to_numpy().astype(float),
                          df_data['Class_Prediction'].to_numpy(),
                          split, list(encoding.keys()), model_config)

  #save the config
  save_params(model_config, model_config['save_folder'] + 'model_config.json')

#remove repeat and sort
def remove_repeat_sort(input_raw):
  #remove repeat
  result = {}
  for key,value in input_raw.items():
    if value not in result.values():
        result[key] = value
  #sort
  keys = list(result.keys())
  values = list(result.values())
  sorted_value_index = np.argsort(values)
  sorted_dict = {keys[i]: values[i] for i in sorted_value_index}
  #return
  return sorted_dict

#plot confusion matrix
def plot_confusion_matrix(labels, predictions, title, ticks, model_config):
  #cmap
  cm = confusion_matrix(labels, predictions)
  #sub
  ax = plt.subplot()
  sns.set(font_scale=2.0) # Adjust to fit
  sns.heatmap(cm, annot=True, ax=ax, cmap="binary", fmt="g");
  # Labels, title and ticks
  label_font = {'size':'20'}  # Adjust to fit
  ax.set_xlabel('Predictions', fontdict=label_font);
  ax.set_ylabel('Class Labels', fontdict=label_font);

  title_font = {'size':'20'}  # Adjust to fit
  ax.set_title(title, fontdict=title_font);

  ax.tick_params(axis='both', which='major', labelsize=8)  # Adjust to fit
  if ticks is not None:
    ax.xaxis.set_ticklabels(ticks);
    ax.yaxis.set_ticklabels(ticks);

  #save
  plt.savefig(model_config['save_folder'] + title + '.png', bbox_inches = "tight")

  #plt.show()
  plt.close()

#analyuze model config
def analyze(model_config):
  ls_metrics = []
  df_col = []
  #append
  ls_metrics.append(model_config['save_folder'])
  df_col.append('save_folder')
  for tag in ['TRAIN', 'VAL']:
    for class_label in (list(model_config['encoding'].keys())+['weighted avg', 'macro avg', 'accuracy']):
      metrics = model_config[tag +'_performance'][class_label]
      #just for accuracy
      if class_label == 'accuracy':
        ls_metrics.append(metrics)
        df_col.append(tag +  ' ' + class_label)
      else:
        #all else
        for metric_type in list(metrics.keys()):
          ls_metrics.append(metrics[metric_type])
          df_col.append(tag +  ' ' + class_label + ' ' + metric_type)
  return ls_metrics, df_col

In [ ]:
#params
sweep_folder = 'AnishSalvi/ImageRx/PAD-Net/ViT_classification/results/3D-ViT-sweep-2023-03-05-20-46-10/'
model_tag = '3D-ViT'

In [ ]:
%%time
#main
if __name__ == '__main__':

  #load
  sweep_config = load_params(sweep_folder + 'sweep_config.json')
  #df
  df = pd.read_pickle(sweep_config['parameters']['data_path']['value'])
  #get the data
  dset_infer = CustomImageDataset(df, sweep_config['parameters']['col_image']['value'])

  #get model paths
  model_dir = sorted(glob.glob(sweep_folder + model_tag + '*'))

  #iterate
  for model_path in model_dir:
    #infer
    save_dfs = []
    #clear
    model, model_config = (None, None)
    if device == 'cuda':
        torch.cuda.empty_cache()
    #load params
    model, model_config = load_pretrained_model(model_path + '/', device)
    #load
    infer_loader = DataLoader(dset_infer, batch_size = model_config['batch_size'])
    #split the df in similar fashion
    ls_df = split_given_size(df, model_config['batch_size'])
    #iterate
    for i, x in enumerate(infer_loader):
      #get the small
      df_small = ls_df[i]
      #pass through pretrained model
      gpu_prediction = model(x.to(device))
      #obtain the individual class predictions
      pred_prob = torch.softmax(gpu_prediction, dim = 1)
      #obtain the class prediction by majority
      pred_class = torch.argmax(pred_prob, dim = 1)
      #class model prediction
      df_small['Class_Prediction'] = pred_class.cpu().detach().numpy()
      #individual class probabilities
      for j in range(model_config['num_classes']):
        pred_tag = 'Class_' + str(j) + '_Prob'
        df_small[pred_tag] = pred_prob.cpu().detach().numpy()[:, j]
      #append
      save_dfs.append(df_small)
      #reset
      x, gpu_prediction, pred_prob, pred_class = (None, None, None, None)
      if device == 'cuda':
        torch.cuda.empty_cache()

    #concat
    df_pred = pd.concat(save_dfs)
    #label if in training or validation
    df_pred['Split'] = df_pred.apply(get_split, axis = 1, args = (model_config,))
    #can perform analysis here later
    get_performance(df_pred, model_config)
    #save the results
    df_pred.to_pickle(model_config['save_folder'] + 'df_results.pkl')

CPU times: total: 3min 37s
Wall time: 1min 52s


In [ ]:
#specify the most relevant columns to keep during analysis
cols = ['save_folder',
        'VAL Control/Non-PAD precision', 'VAL Control/Non-PAD recall', 'VAL Control/Non-PAD f1-score',
        'VAL PAD, <50% stenosis precision', 'VAL PAD, <50% stenosis recall','VAL PAD, <50% stenosis f1-score',
        'VAL PAD, >50% stenosis precision', 'VAL PAD, >50% stenosis recall', 'VAL PAD, >50% stenosis f1-score',

        'VAL weighted avg precision', 'VAL weighted avg recall', 'VAL weighted avg f1-score', 'VAL weighted avg support', 'VAL accuracy'

 ]
#sweep folder
#model tag

In [ ]:
%%time
#load
sweep_config = load_params(sweep_folder + 'sweep_config.json')
#get model paths
model_dir = sorted(glob.glob(sweep_folder + model_tag + '*'))

#performance
performance = []
#iterate
for model_path in model_dir:
  model_config = load_params(model_path + '/' + 'model_config.json')
  #analyze
  model_metrics, df_cols = analyze(model_config)
  #append
  performance.append(model_metrics)
#init
df_sweep = pd.DataFrame(performance, columns = df_cols)
#sort
df_sweep = df_sweep.sort_values(by = ['VAL weighted avg f1-score'], ascending = False, ignore_index = True)
#save
df_sweep.to_excel(sweep_config['parameters']['save_folder']['value'] + 'df_results.xlsx')
#visual
display(df_sweep[cols].round(2))

,save_folder,TRAIN Control/Non-PAD precision,TRAIN Control/Non-PAD recall,TRAIN Control/Non-PAD f1-score,"TRAIN PAD, <50% stenosis precision","TRAIN PAD, <50% stenosis recall","TRAIN PAD, <50% stenosis f1-score","TRAIN PAD, >50% stenosis precision","TRAIN PAD, >50% stenosis recall","TRAIN PAD, >50% stenosis f1-score",VAL weighted avg precision,VAL weighted avg recall,VAL weighted avg f1-score,VAL weighted avg support,VAL accuracy
0,AnishSalvi/ImageRx/PAD-Net/ViT_classification/...,0.73,0.67,0.7,0.35,0.92,0.51,0.88,0.55,0.67,0.43,0.26,0.3,58,0.26


CPU times: total: 281 ms
Wall time: 525 ms


In [ ]:
#remove wandb stuff saved locally
import shutil
shutil.rmtree('wandb')